In [ ]:
!wget https://huggingface.co/datasets/Abirate/english_quotes/resolve/main/quotes.jsonl

--2024-03-08 10:03:59--  https://huggingface.co/datasets/Abirate/english_quotes/resolve/main/quotes.jsonl
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/Abirate/english_quotes/efd879f9583253782a4c424e001b870c1ed698b37e73d80e26d6e7dc3e25935d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27quotes.jsonl%3B+filename%3D%22quotes.jsonl%22%3B&Expires=1710151439&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDE1MTQzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9BYmlyYXRlL2VuZ2xpc2hfcXVvdGVzL2VmZDg3OWY5NTgzMjUzNzgyYTRjNDI0ZTAwMWI4NzBjMWVkNjk4YjM3ZTczZDgwZTI2ZDZlN2RjM2UyNTkzNWQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=HVexyQvS-d7Oh5aAikT2D6KGxYiS6ZYuPAylgV6lFs%7E0

In [ ]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_json('quotes.jsonl', lines=True)

# Display the first few rows of the DataFrame
df.head()

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[be-yourself, gilbert-perreira, honesty, inspi..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[best, life, love, mistakes, out-of-control, t..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[human-nature, humor, infinity, philosophy, sc..."
3,"“So many books, so little time.”",Frank Zappa,"[books, humor]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[books, simile, soul]"


In [ ]:
!pip install accelerate transformers datasets torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["quote"], padding="max_length", truncation=True, max_length=1024)
    outputs = tokenizer(batch["author"], padding="max_length", truncation=True, max_length=1024)

    # create the labels (the model will be trained to predict the labels from the inputs)
    labels = outputs.input_ids.copy()

    # create a mask for the labels (the model will only be trained to predict the tokens after the </s> token)
    labels_mask = [[(i > seq.index(tokenizer.eos_token_id)) for i in range(len(seq))] for seq in inputs.input_ids]
    labels_mask = torch.tensor(labels_mask)

    # replace the tokens before the </s> token with -100 (the model will not be trained to predict these tokens)
    labels = [[(label if mask else -100) for label, mask in zip(seq, mask_seq)] for seq, mask_seq in zip(labels, labels_mask)]

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = labels

    return batch

# convert the DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(process_data_to_model_inputs, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
'''

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["quote"], padding="max_length", truncation=True, max_length=1024)
    outputs = tokenizer(batch["author"], padding="max_length", truncation=True, max_length=1024)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids

    return batch

# convert the DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(process_data_to_model_inputs, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

'''

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate = 0.05,
    num_train_epochs=1,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,

    args=training_args,
    train_dataset=dataset,
)


NameError: name 'model' is not defined

In [ ]:
import sys
print(sys.executable)

/usr/bin/python3


In [ ]:
#!pip uninstall -y accelerate transformers
#!pip install accelerate transformers




# This fixed the dependencies bug

In [ ]:
trainer.train()

Step,Training Loss
500,0.551800
1000,0.000000
1500,0.000000
2000,0.003800
2500,0.003400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=2508, training_loss=0.11142334275458608, metrics={'train_runtime': 2828.9774, 'train_samples_per_second': 0.887, 'train_steps_per_second': 0.887, 'total_flos': 5435098342096896.0, 'train_loss': 0.11142334275458608, 'epoch': 1.0})

In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()
from google.colab import drive
drive.mount('/content/drive/')





Mounted at /content/drive/


In [ ]:

from transformers import BartForConditionalGeneration
import torch

# Initialize the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')  # or any other pretrained BART model

# Load the state dict from the .pth file - gpu will be faster but google colab limits this a lot now!
state_dict = torch.load('/content/drive/MyDrive/Quotes_second.pth', map_location=torch.device('cpu'))

# Update the model's state dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
import os

In [ ]:
#torch.save(model.state_dict(), '/content/Quotes_second.pth')

# copy the file to Google Drive
#!cp /content/Quotes_second.pth /content/drive/

cp: cannot create regular file '/content/drive/Quotes_second.pth': Operation not supported


In [ ]:
!ls -l /content/drive/

total 4
drwx------ 15 root root 4096 Mar  7 17:01 MyDrive


In [ ]:
# save the model
#torch.save(model.state_dict(), '/content/drive/MyDrive/Quotes_second.pth')

In [ ]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [ ]:
from transformers import AutoTokenizer

# Load the pre-trained tokenizer for the BERT base model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize a sentence
input_data = tokenizer.encode("Be yourself; everyone else is already taken", add_special_tokens=True)

# Convert to tensor
input_tensor = torch.tensor([input_data])  # add batch dimension

# Make a prediction
model.eval()


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [ ]:
# Now you can pass the input tensor to the model
with torch.no_grad():
    output_tokens = model.generate(input_tensor, max_length=80)

# input tokens
print(input_tensor)

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(output_text)

NameError: name 'input_tensor' is not defined

In [ ]:
from transformers import BartTokenizer

# Initialize the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# List of strings to generate output for
strings = ["Where is the chocolate?"]

# Loop through the strings
for s in strings:
    # Tokenize the string and convert to tensor
    input_tensor = tokenizer.encode(s, return_tensors='pt')

    # Generate output
    with torch.no_grad():
        output_tokens = model.generate(input_tensor, max_length=40)

    # Decode the output tokens
    output_string = tokenizer.decode(output_tokens[0])

    print(output_string)

</s><s><pad><pad><pad>umi<pad><pad>iatus<pad><pad> Hebdo<pad><pad>Override<pad><pad>romancer<pad><pad> Summoner<pad><pad>reditary<pad><pad>elist<pad><pad>ictionary<pad><pad>Filter<pad><pad> ACPI<pad><pad>afort</s>


In [ ]:
output_string_cleaned = output_string.replace('<s>', '').replace('</s>', '').replace('<pad>', '')
print(output_string_cleaned)

umiiatus HebdoOverrideromancer SummonerreditaryelistictionaryFilter ACPIafort


In [ ]:
# Assuming that `device` is your GPU device...
device = torch.device('cuda:0')

# Move the model to the GPU
model = model.to(device)

# Move the input tensor to the GPU
input_tensor = input_tensor.to(device)

# Now you can pass the input tensor to the model
with torch.no_grad():
    output_tokens = model.generate(input_tensor, max_length=80)

In [ ]:
print(input_data)
print(input_tensor)

with torch.no_grad():
    # Generate a sequence of tokens
    output_tokens = model.generate(input_tensor, max_length=40)

# Decode the output tokens
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Print the output text
print(output_text)



#decoded_text = tokenizer.decode([101, 4789, 15313, 102])
#print(decoded_text)

[101, 2022, 4426, 1025, 3071, 2842, 2003, 2525, 2579, 102]
tensor([[ 101, 2022, 4426, 1025, 3071, 2842, 2003, 2525, 2579,  102]])
[unused1] [unused0] [unused0] extraction [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused1]
[CLS] albert einstein [SEP]


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Tokenize a sentence
input_data = tokenizer.encode("Be yourself; everyone else is already taken", add_special_tokens=True)

# Convert to tensor
input_tensor = torch.tensor([input_data])  # add batch dimension

# Move the model and input tensor to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
input_tensor = input_tensor.to(device)

# Make a prediction
model.eval()
with torch.no_grad():
    output = model(input_tensor)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import BertModel

# Initialize a model
model = BertModel.from_pretrained('bert-base-uncased')

# Load the state dictionary
state_dict = torch.load('/content/drive/MyDrive/Quotes_second.pth')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for BertModel:
	Missing key(s) in state_dict: "embeddings.word_embeddings.weight", "embeddings.position_embeddings.weight", "embeddings.token_type_embeddings.weight", "embeddings.LayerNorm.weight", "embeddings.LayerNorm.bias", "encoder.layer.0.attention.self.query.weight", "encoder.layer.0.attention.self.query.bias", "encoder.layer.0.attention.self.key.weight", "encoder.layer.0.attention.self.key.bias", "encoder.layer.0.attention.self.value.weight", "encoder.layer.0.attention.self.value.bias", "encoder.layer.0.attention.output.dense.weight", "encoder.layer.0.attention.output.dense.bias", "encoder.layer.0.attention.output.LayerNorm.weight", "encoder.layer.0.attention.output.LayerNorm.bias", "encoder.layer.0.intermediate.dense.weight", "encoder.layer.0.intermediate.dense.bias", "encoder.layer.0.output.dense.weight", "encoder.layer.0.output.dense.bias", "encoder.layer.0.output.LayerNorm.weight", "encoder.layer.0.output.LayerNorm.bias", "encoder